This notebook is a sample code with Japanese comments.

# 2.5 勾配ブースティングが最強？！　いろいろな機械学習アルゴリズムを使ってみよう

In [2]:
import numpy as np
import pandas as pd

train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')
gender_submission = pd.read_csv('../input/titanic/gender_submission.csv')

data = pd.concat([train, test], sort=False)

data['Sex'].replace(['male', 'female'], [0, 1], inplace=True)
data['Embarked'].fillna(('S'), inplace=True)
data['Embarked'] = data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)
data['Fare'].fillna(np.mean(data['Fare']), inplace=True)
data['Age'].fillna(data['Age'].median(), inplace=True)
data['FamilySize'] = data['Parch'] + data['SibSp'] + 1
data['IsAlone'] = 0
data.loc[data['FamilySize'] == 1, 'IsAlone'] = 1

In [3]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsAlone
0,1,0.0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0,2,0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1,2,0
2,3,1.0,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,1,1
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0,2,0
4,5,0.0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0,1,1


In [4]:
delete_columns = ['Name', 'PassengerId', 'Ticket', 'Cabin']
data.drop(delete_columns, axis=1, inplace=True)

train = data[:len(train)]
test = data[len(train):]

y_train = train['Survived']
X_train = train.drop('Survived', axis=1)
X_test = test.drop('Survived', axis=1)

In [5]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone
0,3,0,22.0,1,0,7.2500,0,2,0
1,1,1,38.0,1,0,71.2833,1,2,0
2,3,1,26.0,0,0,7.9250,0,1,1
3,1,1,35.0,1,0,53.1000,0,2,0
4,3,0,35.0,0,0,8.0500,0,1,1


# sklearn

In [6]:
from sklearn.linear_model import LogisticRegression

# ロジスティック回帰
clf = LogisticRegression(penalty='l2', solver='sag', random_state=0)

In [7]:
from sklearn.ensemble import RandomForestClassifier

# ランダムフォレスト
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

In [8]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [9]:
y_pred[:10]

array([0., 1., 0., 0., 1., 0., 1., 0., 1., 0.])

In [10]:
sub = pd.read_csv('../input/titanic/gender_submission.csv')

In [11]:
sub['Survived'] = list(map(int, y_pred))
sub.to_csv('submission_randomforest.csv', index=False)
sub.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


# LightGBM

In [15]:
!pip install lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 40.7 MB/s eta 0:00:00a 0:00:01


In [16]:
from sklearn.model_selection import train_test_split

# 勾配ブースティングと呼ばれる方法
# 大量の決定木を作成しながら学習を進める
# 過学習対策のために、検証用データに対する性能を見ながら学習を打ち切る「early stopping」を利用するのが一般的

# 学習用と検証用にデータセットを分割
X_train, X_valid, y_train, y_valid = \
    train_test_split(X_train, y_train, test_size=0.3,
                     random_state=0, stratify=y_train)

In [17]:
# カテゴリ変数をリスト形式で宣言
# LightGBMは、カテゴリ変数に対して特別な処理を自動的に実行してくれる
categorical_features = ['Embarked', 'Pclass', 'Sex']

In [18]:
import lightgbm as lgb


lgb_train = lgb.Dataset(X_train, y_train,
                        categorical_feature=categorical_features)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train,
                       categorical_feature=categorical_features)

params = {
    'objective': 'binary'
}

model = lgb.train(params, lgb_train,
                  valid_sets=[lgb_train, lgb_eval],
                  # 学習の実行回数の最大値
                  num_boost_round=1000,
                  callbacks=[
                      lgb.early_stopping(
                          # early stoppingの判定基準
                          # 連続して10回学習を重ねても検証用データセットに対する性能が改善しなかった場合に学習を打ち切る
                          stopping_rounds=10,
                          verbose=True
                      ),
                      lgb.log_evaluation(10)
                  ]
                 )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

[LightGBM] [Info] Number of positive: 167, number of negative: 269
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002404 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 172
[LightGBM] [Info] Number of data points in the train set: 436, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383028 -> initscore=-0.476718
[LightGBM] [Info] Start training from score -0.476718
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Wa

In [19]:
y_pred[:10]

array([0.14665841, 0.42711079, 0.17556106, 0.05032947, 0.30673957,
       0.35161146, 0.69605657, 0.29078791, 0.69132888, 0.04533161])

In [20]:
# 出力結果が0~1の連続値なので、0.5を超えていれば1と予測したとみなす
y_pred = (y_pred > 0.5).astype(int)
y_pred[:10]

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0])

In [21]:
sub['Survived'] = y_pred
sub.to_csv('submission_lightgbm.csv', index=False)

sub.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
